# Portal for ArcGIS Logs

- accessing the logs through the Portal Admin Directory

In [1]:
import json
from urllib.parse import urljoin

import requests

In [2]:
def get_credentials():
    '''Get the username and password for the Server sites in a Dict with keys: 'user' and 'pswd'
    '''

    with open('params.json', 'r') as jsonfile:
        jsontxt = jsonfile.read()

        creds_json = json.loads(jsontxt)

    user = creds_json['login']['username']
    pswd = creds_json['login']['password']

    creds = {
        "user": user,
        "pswd": pswd
        }

    return creds

## Get the Portal Admin Directory URL

In [3]:
def get_portaladmin_url(portal):
    '''Returns the Portal Administrator Directory URL for a Portal
    '''

    # get Portal credentials and URL
    # stored in a dict in a json file stored outside version control

    with open('params.json', 'r') as jsonfile:
        jsontxt = jsonfile.read()

        creds = json.loads(jsontxt)

    # get the URL of the Portal Administrator Directory API root resource
    # https://developers.arcgis.com/rest/enterprise-administration/portal/portal-administration-root/


    root_url = creds['portal'][portal]['admin']

    return root_url

## Generate a Token

- Any of the functions that operate on a password-protected REST API require a token for access
- Best practice in scripting is to generate a single token with the function below, then pass it as an argument to later functions.
- Each function has the ability to generate its own token, if none is passed.


In [4]:
def get_token(portal, exp_min=90):
    '''Returns a token that can be used by clients when working with the Portal Admin API.
    '''

    # get Portal credentials, stored outside version control
    creds = get_credentials()

    # build the URL to the 'Generate Token' endpoint
    token_url = rf"https://{portal}.udot.utah.gov/portal/sharing/rest/generateToken/"

    # set the payload for the HTTP POST request
    # default expiration:  90 minutes

    payload = {
        "username": creds['user'],
        "password": creds['pswd'],
        "client": "requestip",
        "expiration": exp_min,
        "f": "json"
        }

    # send the POST requests using the Requests library
    r = requests.post(token_url, data=payload)

    # parse the token value out of the response JSON
    token = r.json()["token"]

    return token

## Root Admin Directory

In [ ]:
portalname = "roads"
url = get_portaladmin_url(portalname)

token = get_token(portalname)

In [ ]:
print(url)
print(token)


'L-AMogPppjTHWNvfljIju0Q1aodEeqd-up2G2wwlY4jWjnyaV6ZxsxWXhamzUjd8pUF5oVKu7X1-BYJa0egJ2xvgnNpWk5pxkcf7Bx4n4LYkaT6_NQPGVw3zH_aHufgbrLz7RC2Qa5MliHV19kmOoXDzKgbvXep-UPf_JoCP1ls.'

In [ ]:

print(url)

payload = {
    'token': token,
    'f': 'pjson'
    }

r = requests.get(url, params=payload)

print(r.url)
print(r.json())

https://roads.udot.utah.gov/portal/portaladmin/
https://roads.udot.utah.gov/portal/portaladmin/?token=L-AMogPppjTHWNvfljIju0Q1aodEeqd-up2G2wwlY4jWjnyaV6ZxsxWXhamzUjd8pUF5oVKu7X1-BYJa0egJ2-WrMNoUIm2grR2NJpMGU1beiAQjn20fCOzVCGEek5rrc93Fj6DGaGbeo9B7SmFL4tLCpfLRT3z-2acjQeiW7eA.&f=pjson
{'error': {'code': 498, 'message': 'Invalid Token.', 'details': ['Token would have expired, regenerate token and send the request again.', 'If the token is generated based on the referrer make sure the referrer information is available with every request in header.']}}
